### Stop Words

#### Task description
Improve the previous program to calculate how many stop words are in the input dataset. Stop words list is in ‘/datasets/stop_words_en.txt’ file. Use Hadoop counter to count the number of stop words and total words in the dataset. The result is the percentage of stop words in the entire dataset (without percent symbol).

In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

path_to_file='stop_words_en.txt'

with open(path_to_file) as stop_words_file:
    content = stop_words_file.readlines()
    stop_words = set(l.strip().lower() for l in content)

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "all\t%d" % 1
        
        if word in stop_words:
            print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % 1
            print "stop_words\t%d" % 1

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys

result = {'all': 0, 'stop_words':0}

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    
    if key:
        result[key] = result[key] + count

all_words = result['all']
stop_words = result['stop_words']
persentage = float(stop_words) / all_words * 100
print persentage

Overwriting reducer.py


In [3]:
%%bash

OUT_DIR="stop_words_result"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming stop words" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/part-00000 | head

38.4403690091	


rm: `stop_words_result': No such file or directory
18/01/14 19:19:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/14 19:19:16 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/14 19:19:16 INFO mapred.FileInputFormat: Total input files to process : 1
18/01/14 19:19:16 INFO mapreduce.JobSubmitter: number of splits:2
18/01/14 19:19:16 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1515912325866_0014
18/01/14 19:19:16 INFO impl.YarnClientImpl: Submitted application application_1515912325866_0014
18/01/14 19:19:16 INFO mapreduce.Job: The url to track the job: http://4c3b86566330:8088/proxy/application_1515912325866_0014/
18/01/14 19:19:16 INFO mapreduce.Job: Running job: job_1515912325866_0014
18/01/14 19:19:55 INFO mapreduce.Job: Job job_1515912325866_0014 running in uber mode : false
18/01/14 19:19:55 INFO mapreduce.Job:  map 0% reduce 0%
18/01/14 19:20:11 INFO mapreduce.Job:  map 32% reduce 0%
18/01/14 19:20:17 INFO mapred